In [ ]:
# approximately 10B tokens
CHARS_NEEDED = 72_000_000_000
CHARS_NEEDED_VAL = 100_000_000

In [ ]:
from datasets import load_dataset, concatenate_datasets
from tqdm.auto import tqdm

from pathlib import Path
train_path = Path("large/train.txt")
val_path = Path("large/val.txt")

We are using downloaded dataset https://huggingface.co/datasets/HuggingFaceTB/dclm-edu

In [1]:
!ls /mnt/harddrive/datasets/dclm-edu/data/

000_00000.parquet  006_00000.parquet  012_00000.parquet  018_00000.parquet
001_00000.parquet  007_00000.parquet  013_00000.parquet  019_00000.parquet
002_00000.parquet  008_00000.parquet  014_00000.parquet  020_00000.parquet
003_00000.parquet  009_00000.parquet  015_00000.parquet  64446_stats.json
004_00000.parquet  010_00000.parquet  016_00000.parquet
005_00000.parquet  011_00000.parquet  017_00000.parquet


In [7]:
concat_ds = None
for i in range(10):
    chunk_path = f"/mnt/harddrive/datasets/dclm-edu/data/0{i:02d}_00000.parquet"

    ds = load_dataset("parquet", data_files=str(chunk_path), split="train")  # Arrow-backed, zero-copy-ish
    ds = ds.filter(lambda ex: ex["metadata"]["edu_score"] > 2.75, num_proc=4, load_from_cache_file=False)
    ds = ds.map(lambda ex: {"text": ex["text"] + "<|endoftext|>"}, num_proc=4, load_from_cache_file=False)
    if concat_ds is None:
        concat_ds = ds
    else:
        concat_ds = concatenate_datasets([concat_ds, ds])

Filter (num_proc=4): 2920000 examples [00:03, 376598.46 examples/s]          
Map (num_proc=4): 2064694 examples [00:11, 88818.79 examples/s]           
Filter (num_proc=4): 2916000 examples [00:04, 359415.00 examples/s]          
Map (num_proc=4): 100%|██████████| 1035062/1035062 [00:11<00:00, 88627.23 examples/s]
Generating train split: 1458000 examples [00:32, 45054.08 examples/s]
Map (num_proc=4): 100%|██████████| 1034976/1034976 [00:12<00:00, 84372.57 examples/s]
Generating train split: 1471000 examples [00:32, 45334.88 examples/s]
Map (num_proc=4): 100%|██████████| 1043177/1043177 [00:13<00:00, 76863.09 examples/s]
Generating train split: 1464000 examples [00:32, 45243.84 examples/s]
Map (num_proc=4): 100%|██████████| 1036961/1036961 [00:11<00:00, 88814.80 examples/s] 
Generating train split: 1463000 examples [00:33, 44066.25 examples/s]
Map (num_proc=4): 100%|██████████| 1033608/1033608 [00:11<00:00, 89549.96 examples/s] 
Generating train split: 1461000 examples [00:32, 44864.21

In [11]:
val_handle = val_path.open("w")
train_handle = train_path.open("w")

In [ ]:
# Write data to txt in batches

listout = []
i = 0
chars_processed = 0
for row in tqdm(concat_ds):
    listout.append(row["text"])
    if chars_processed <= CHARS_NEEDED_VAL:
        batch = 100
        if len(listout) > batch:
            _ = val_handle.write("".join(listout))
            listout = []
    elif CHARS_NEEDED_VAL < chars_processed <= CHARS_NEEDED:
        if len(listout) > 1000:
            _ = train_handle.write("".join(listout))
            listout = []
    elif chars_processed > CHARS_NEEDED:
        break

    chars_processed += len(row["text"])
    i += 1

100%|██████████| 10361335/10361335 [05:03<00:00, 34091.95it/s]


In [14]:
train_handle.close()
val_handle.close()

In [16]:
!ls -lh large

total 58G
-rw-rw-r-- 1 george george 58G Oct  5 00:59 train.txt
-rw-rw-r-- 1 george george 96M Oct  5 00:54 val.txt
